In [1]:
import pandas as pd
import omnipath as op
import os

/home/dbdimitrov/miniconda3/envs/spiana/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Get Reference/Source Info From OmniPath

In [2]:
ligrec = op.interactions.import_intercell_network()
ligrec.head()

ligrec = ligrec.rename(columns={'genesymbol_intercell_source':'ligand', 'genesymbol_intercell_target':'receptor'})
ligrec = ligrec[['ligand', 'receptor', 'references'] + [col for col in ligrec.columns if col not in ['ligand', 'receptor', 'references']]]
ligrec.head()

,ligand,receptor,references,source,target,is_stimulation,is_inhibition,consensus_direction,consensus_stimulation,consensus_inhibition,...,aspect_intercell_target,category_source_intercell_target,uniprot_intercell_target,entity_type_intercell_target,consensus_score_intercell_target,transmitter_intercell_target,receiver_intercell_target,secreted_intercell_target,plasma_membrane_transmembrane_intercell_target,plasma_membrane_peripheral_intercell_target
0,CALM1,TRPC1,TRIP:11290752;TRIP:11983166;TRIP:12601176,P0DP23,P48995,False,True,True,False,True,...,functional,resource_specific,P48995,protein,2,False,True,False,False,False
1,CALM1,TRPC1,TRIP:11290752;TRIP:11983166;TRIP:12601176,P0DP23,P48995,False,True,True,False,True,...,functional,resource_specific,P48995,protein,3,False,True,False,False,False
2,CALM1,TRPC1,TRIP:11290752;TRIP:11983166;TRIP:12601176,P0DP23,P48995,False,True,True,False,True,...,functional,composite,P48995,protein,3,False,True,False,False,False
3,CALM1,TRPC1,TRIP:11290752;TRIP:11983166;TRIP:12601176,P0DP23,P48995,False,True,True,False,True,...,functional,resource_specific,P48995,protein,2,False,True,False,False,False
4,CALM1,TRPC1,TRIP:11290752;TRIP:11983166;TRIP:12601176,P0DP23,P48995,False,True,True,False,True,...,functional,resource_specific,P48995,protein,3,False,True,False,False,False


In [3]:
ligrec = ligrec[['source', 'target', 'sources', 'references']]

LIANA resource

In [4]:
full_resource = pd.read_csv('https://raw.githubusercontent.com/saezlab/liana-py/main/liana/resource/omni_resource.csv', index_col=0)
resource = full_resource[full_resource['resource'] == 'consensus']
resource = resource.rename(columns={'source_genesymbol': 'ligand',
                                    'target_genesymbol': 'receptor'}
                            )
resource = resource.drop('resource', axis=1)

In [5]:
resource = resource.merge(ligrec, how='left', on=['source', 'target']).drop_duplicates().sort_values(['source', 'target'])

In [6]:
missing_source = resource[resource['sources'].isna()][['source', 'target']]

In [7]:
sources = full_resource[~full_resource['resource'].isin(['consensus', 'mouseconsensus'])].merge(missing_source, how='inner')
sources = sources[['source', 'target', 'resource']].rename({'resource':'sources'}, axis=1)
# collaps source
found_sources = sources.groupby(['source', 'target']).agg({'sources': ','.join}).reset_index()

In [8]:
resource = pd.merge(resource, found_sources, on=['source', 'target'], how='left', suffixes=('', '_from_df2'))

# Fill missing 'sources' values in df1 with those from df2
resource['sources'] = resource['sources'].fillna(resource['sources_from_df2'])

# Drop the temporary 'sources_from_df2' column
resource.drop(columns=['sources_from_df2'], inplace=True)

In [9]:
resource.head()

,source,target,ligand,receptor,sources,references
0,A6NMZ7,COMPLEX:O75578_P05556,COL6A6,ITGA10_ITGB1,CellChatDB;CellPhoneDB;ICELLNET,ICELLNET:15147720;ICELLNET:16988024;ICELLNET:2...
1,A6NMZ7,COMPLEX:P05556_P17301,COL6A6,ITGA2_ITGB1,CellChatDB;CellPhoneDB;ICELLNET,ICELLNET:15147720;ICELLNET:16988024;ICELLNET:2...
2,A6NMZ7,COMPLEX:P05556_P26006,COL6A6,ITGA3_ITGB1,CellChatDB,NaN
3,A6NMZ7,COMPLEX:P05556_P56199,COL6A6,ITGA1_ITGB1,CellPhoneDB;ICELLNET,ICELLNET:15147720;ICELLNET:16988024;ICELLNET:2...
4,A6NMZ7,COMPLEX:P05556_Q13797,COL6A6,ITGA9_ITGB1,CellChatDB,NaN


In [10]:
resource.to_csv(os.path.join("..", "..", "figures", "source", "LigandReceptorPK.csv"))